# Training of model for identification of molecular structure from mass spectrum
This notebook demonstrates the training and identification of our models.
Before running this notebook, you need to run DataFetch.ipynb Preprocess.ipynb train.ipynb

In [1]:
from pcemg.scripts.ms_train import ms_train
!cat model_config.ini
ms_train("./MS_vocab.txt","./massbank.pkl","./vae_model/model.iter-160000",'model_config.ini')

RDKit WARNING: [04:41:42] Enabling RDKit 2019.09.3 jupyter extensions


[JTVAE]
hidden_size = 100
latent_size = 56
depthT = 20
depthG = 5

[PEAK_ENCODER]
max_mpz = 1000
embedding_size = 10
conv1_channel = 64
kernel1_width = 5
hidden_size = 200
num_rnn_layers = 2
bidirectional = False
output_size = 56
dropout_rate = 0.2
use_batchnorm=False

[TRAINING]
; Number of epoch to finish learning
max_epoch = 300
; Number of epoch to start learning the decoder
fine_tunning_warmup = 30

;*********************************************;
;  weight parameters
;  1. word                = word_rate
;  2. topology            = topo_rate
;  3. assemble            = assm_rate
;  4. KLDivergence        = beta
;  5. L2 regularization   = reg_rate
;
;  total loss =
;       word_rate * word_loss + 
;       topo_rate * topo_loss + 
;       assm_rate * assm_loss +
;       beta * kl_loss +
;       reg_rate * l2_reg
;*********************************************;
; weight of Word loss
word_rate = 1
; weight of topology loss
topo_rate = 1
; weight of assemble loss
assm_rate = 1
; weight

/home/kusachi/anaconda3/envs/workspace/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


JTNNVAE(
  (jtnn): JTNNEncoder(
    (embedding): Embedding(1057, 100)
    (outputNN): Sequential(
      (0): Linear(in_features=200, out_features=100, bias=True)
      (1): ReLU()
    )
    (GRU): GraphGRU(
      (W_z): Linear(in_features=200, out_features=100, bias=True)
      (W_r): Linear(in_features=100, out_features=100, bias=False)
      (U_r): Linear(in_features=100, out_features=100, bias=True)
      (W_h): Linear(in_features=200, out_features=100, bias=True)
    )
  )
  (decoder): JTNNDecoder(
    (embedding): Embedding(1057, 100)
    (W_z): Linear(in_features=200, out_features=100, bias=True)
    (U_r): Linear(in_features=100, out_features=100, bias=False)
    (W_r): Linear(in_features=100, out_features=100, bias=True)
    (W_h): Linear(in_features=200, out_features=100, bias=True)
    (W): Linear(in_features=128, out_features=100, bias=True)
    (U): Linear(in_features=128, out_features=100, bias=True)
    (U_i): Linear(in_features=200, out_features=100, bias=True)
    (W_o)

9 epoch[3290] kl_loss 285.14, Word: 45.24, Topo: 84.71, Assm: 79.10 vali_Word: 52.06, vali_Topo: 86.40, vali_assm: 82.41 
19 epoch[6580] kl_loss 285.93, Word: 53.20, Topo: 87.04, Assm: 80.96 vali_Word: 55.34, vali_Topo: 87.43, vali_assm: 84.08 
29 epoch[9870] kl_loss 305.24, Word: 57.66, Topo: 88.53, Assm: 81.81 vali_Word: 55.76, vali_Topo: 88.01, vali_assm: 83.70 
39 epoch[13160] kl_loss 253.51, Word: 73.43, Topo: 95.65, Assm: 91.19 vali_Word: 71.20, vali_Topo: 94.93, vali_assm: 89.47 
49 epoch[16450] kl_loss 233.89, Word: 78.51, Topo: 97.41, Assm: 94.29 vali_Word: 72.28, vali_Topo: 95.29, vali_assm: 88.45 
59 epoch[19740] kl_loss 238.48, Word: 81.83, Topo: 98.11, Assm: 95.15 vali_Word: 73.12, vali_Topo: 95.45, vali_assm: 89.58 
69 epoch[23030] kl_loss 240.36, Word: 84.49, Topo: 98.55, Assm: 95.98 vali_Word: 72.63, vali_Topo: 95.72, vali_assm: 90.32 
79 epoch[26320] kl_loss 243.98, Word: 86.47, Topo: 98.79, Assm: 96.40 vali_Word: 72.22, vali_Topo: 95.66, vali_assm: 89.27 
89 epoch[296

In [1]:
from pcemg.scripts.analysis_model import analysisModel
analysisModel('./result-20200622-0441.tar.gz',eval_mode=True)

RDKit WARNING: [01:01:32] Enabling RDKit 2019.09.3 jupyter extensions




[JTVAE]
hidden_size = 100
latent_size = 56
depthT = 20
depthG = 5

[PEAK_ENCODER]
max_mpz = 1000
embedding_size = 10
conv1_channel = 64
kernel1_width = 5
hidden_size = 200
num_rnn_layers = 2
bidirectional = False
output_size = 56
dropout_rate = 0.2
use_batchnorm=False

[TRAINING]
; Number of epoch to finish learning
max_epoch = 300
; Number of epoch to start learning the decoder
fine_tunning_warmup = 30

;*********************************************;
;  weight parameters
;  1. word                = word_rate
;  2. topology            = topo_rate
;  3. assemble            = assm_rate
;  4. KLDivergence        = beta
;  5. L2 regularization   = reg_rate
;
;  total loss =
;       word_rate * word_loss + 
;       topo_rate * topo_loss + 
;       assm_rate * assm_loss +
;       beta * kl_loss +
;       reg_rate * l2_reg
;*********************************************;
; weight of Word loss
word_rate = 1
; weight of topology loss
topo_rate = 1
; weight of assemble loss
assm_rate = 1
; weig

/home/kusachi/anaconda3/envs/workspace/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


False
Structuer of Encoder
 ms_peak_encoder_cnn(
  (embedding): Embedding(1000, 10)
  (convSequential): Sequential(
    (conv1-1): conv_set(
      (convSequential): Sequential(
        (0): Conv1d(11, 64, kernel_size=(5,), stride=(1,), padding=(2,), padding_mode=zero)
        (1): LeakyReLU(negative_slope=0.01)
      )
    )
    (conv1-2): conv_set(
      (convSequential): Sequential(
        (0): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), padding_mode=zero)
        (1): LeakyReLU(negative_slope=0.01)
      )
    )
    (conv1-3): conv_set(
      (convSequential): Sequential(
        (0): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), padding_mode=zero)
        (1): LeakyReLU(negative_slope=0.01)
      )
    )
    (transpose1): Transpose()
    (dropout1): Dropout(p=0.2, inplace=False)
    (gru): GRU(64, 200, num_layers=2, batch_first=True)
    (gru-output): GRUOutput()
    (dropout2): Dropout(p=0.2, inplace=False)
  )
  (sampling): Sampling(
    (var): Line

please choice iterate [3290 6580 ... 95410 98700] >>  last



720
17
[0.35       0.64       0.83783784 0.8        1.         0.85
 0.74285714 0.65517241 1.         0.93333333 0.58974359 0.53333333
 0.625      0.52941176 0.54545455 0.96428571 0.83333333 0.94444444
 0.65       1.         0.5        0.34782609 1.         0.64705882
 0.85714286 0.58333333 0.84       0.72727273 0.51428571 0.8
 0.75       1.         0.38461538 1.         0.68181818 1.
 1.         0.33333333 0.86956522 0.21621622 0.46428571 0.25714286
 0.48648649 0.27272727 0.4        0.77272727 1.         0.64516129
 0.51515152 1.         1.         0.73684211 0.62162162 0.46428571
 0.93333333 0.71428571 0.51428571 0.625      0.6        0.6
 0.63157895 0.76470588 0.48148148 0.88235294 0.85185185 0.75862069
 1.         0.68181818 0.91428571 0.48837209 0.78571429 1.
 1.         0.54761905 1.         1.         0.875      0.6
 0.92       0.91304348 0.47058824 1.         0.57692308 0.39285714
 1.         0.5862069  0.81578947 0.57894737 1.         1.
 0.8        0.3559322  1.         0.41

In [1]:
from pcemg.scripts.saliency_calc import saliencyCalc

saliencyCalc("./result-20200622-0441.tar.gz")

RDKit WARNING: [08:54:22] Enabling RDKit 2019.09.3 jupyter extensions


[JTVAE]
hidden_size = 100
latent_size = 56
depthT = 20
depthG = 5

[PEAK_ENCODER]
max_mpz = 1000
embedding_size = 10
conv1_channel = 64
kernel1_width = 5
hidden_size = 200
num_rnn_layers = 2
bidirectional = False
output_size = 56
dropout_rate = 0.2
use_batchnorm=False

[TRAINING]
; Number of epoch to finish learning
max_epoch = 300
; Number of epoch to start learning the decoder
fine_tunning_warmup = 30

;*********************************************;
;  weight parameters
;  1. word                = word_rate
;  2. topology            = topo_rate
;  3. assemble            = assm_rate
;  4. KLDivergence        = beta
;  5. L2 regularization   = reg_rate
;
;  total loss =
;       word_rate * word_loss + 
;       topo_rate * topo_loss + 
;       assm_rate * assm_loss +
;       beta * kl_loss +
;       reg_rate * l2_reg
;*********************************************;
; weight of Word loss
word_rate = 1
; weight of topology loss
topo_rate = 1
; weight of assemble loss
assm_rate = 1
; weight

/home/kusachi/anaconda3/envs/workspace/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


False
Structuer of Encoder
 ms_peak_encoder_cnn(
  (embedding): Embedding(1000, 10)
  (convSequential): Sequential(
    (conv1-1): conv_set(
      (convSequential): Sequential(
        (0): Conv1d(11, 64, kernel_size=(5,), stride=(1,), padding=(2,), padding_mode=zero)
        (1): LeakyReLU(negative_slope=0.01)
      )
    )
    (conv1-2): conv_set(
      (convSequential): Sequential(
        (0): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), padding_mode=zero)
        (1): LeakyReLU(negative_slope=0.01)
      )
    )
    (conv1-3): conv_set(
      (convSequential): Sequential(
        (0): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), padding_mode=zero)
        (1): LeakyReLU(negative_slope=0.01)
      )
    )
    (transpose1): Transpose()
    (dropout1): Dropout(p=0.2, inplace=False)
    (gru): GRU(64, 200, num_layers=2, batch_first=True)
    (gru-output): GRUOutput()
    (dropout2): Dropout(p=0.2, inplace=False)
  )
  (sampling): Sampling(
    (var): Line

please choice iterate [3290 6580 ... 95410 98700] >>  last


100%|██████████| 730/730 [41:48<00:00,  3.44s/it]   
